In [57]:
import sympy as sp
import numpy as np

# Extracting bus data and branch data
Bus_data = np.loadtxt('bus.txt', delimiter=' ', skiprows=1, dtype=float)
branch_data = np.loadtxt('branch.txt', delimiter=' ', skiprows=1, dtype=float)

# Finding the number of buses
n = Bus_data.shape[0]

# Finding the number of branches present
m = branch_data.shape[0]

# Initialize zero matrix for impedance
b = np.zeros((n, n))
# Replace zero matrix elements with actual ones
for i in range(m):
    j = int(branch_data[i, 0]) - 1
    k = int(branch_data[i, 1]) - 1
    b[j, k] = 100 * -branch_data[i, 2]  # To keep the power injections in MW, we multiply [B] by 100
    b[k, j] = 100 * -branch_data[i, 2]
    b[k, k] += 100 * branch_data[i, 2]
    b[j, j] += 100 * branch_data[i, 2]

# Convert b to a SymPy matrix
b = sp.Matrix(b)

# Define variables
P1, P2, P3, λ1, λ2, λ3, λ4, mu1, θ1, θ2, θ3 = sp.symbols('P1 P2 P3 λ1 λ2 λ3 λ4 mu1 θ1 θ2 θ3')

thetas = [θ1, θ2, θ3]
lamda=[λ1, λ2, λ3]
Power=[P1, P2, P3]

# Define the cost functions of each generator
f1 = 561 + 7.92 * P1 + 0.001562 * P1**2
f2 = 310 + 7.85 * P2 + 0.00194 * P2**2
f3 = 78 + 7.97 * P3 + 0.00482 * P3**2

# Initialize zero matrix for OPF using SymPy
g = sp.zeros(n, 1)

# Define inequality constraints

h= P1-300

print(h)
# Define the constraint to make 1st bus to be slack
s=θ1-0
# Define the constraint function from OPF
for i in range(n):
    for k in range(n):
            g[i] += b[i, k] * thetas[k]

Load=0
for i in range(n):
    g[i] += -Power[i] + Bus_data[i, 2]
    Load += Bus_data[i, 2]

Cost_function = f1+f2+f3
print(Cost_function)

# Define the Lagrangian
for i in range(n):
    Cost_function += lamda[i] * g[i]

L=Cost_function
L +=  λ4*s

print("Lagrange_function=",L)

# Compute the partial derivatives

# Compute the gradient of the Lagrangian

LP1 = sp.diff(L, P1)
LP2 = sp.diff(L, P2)
LP3 = sp.diff(L, P3)
Lθ1 = sp.diff(L, θ1)
Lθ2 = sp.diff(L, θ2)
Lθ3 = sp.diff(L, θ3)
Lλ1 = sp.diff(L, λ1)
Lλ2 = sp.diff(L, λ2)
Lλ3 = sp.diff(L, λ3)
Lλ4 = sp.diff(L, λ4)
print(f"LP1 = {LP1}\nLP2 = {LP2}\nLP3 = {LP3}\nLθ1 = {Lθ1}\nLθ2 = {Lθ2}\nLθ3 = {Lθ3}\nLλ1 = {Lλ1}\nLλ2 = {Lλ2}\nLλ3 = {Lλ3}\nLλ4 = {Lλ4}")# Solve the system of equations


#Define number of inequlaity constrain


solution = sp.solve([LP1, LP2,LP3, Lλ1,Lλ2,Lλ3,Lλ4, Lθ1,Lθ2,Lθ3], (P1, P2, P3,λ1,λ2,λ3,λ4, θ1,θ2,θ3))

# Check the Inequality Constraints

if solution[P1]<= 300:
    solution = sp.solve([LP1, LP2, LP3, Lλ1, Lλ2, Lλ3, Lλ4, Lθ1, Lθ2, Lθ3], (P1, P2, P3, λ1, λ2, λ3, λ4, θ1, θ2, θ3))
    print("Generator values are within limit")
    print("Without generator limit implementation",solution)

else:
  L += mu1*(P1-400)
LP1 = sp.diff(L, P1)
LP2 = sp.diff(L, P2)
LP3 = sp.diff(L, P3)
Lθ1 = sp.diff(L, θ1)
Lθ2 = sp.diff(L, θ2)
Lθ3 = sp.diff(L, θ3)
Lλ1 = sp.diff(L, λ1)
Lλ2 = sp.diff(L, λ2)
Lλ3 = sp.diff(L, λ3)
Lλ4 = sp.diff(L, λ4)
Lmu1= P1-300
 # Define your equations
equations = [LP1, LP2, LP3, Lλ1, Lλ2, Lλ3, Lλ4, Lmu1, Lθ1, Lθ2, Lθ3]

# Solve the system of equations
solution = sp.solve(equations, (P1, P2, P3, λ1, λ2, λ3, λ4, mu1, θ1, θ2, θ3))

# Print the solution
print("With generator limit implementation", solution)

angle=(solution[θ1], solution[θ2], solution[θ3])

# Calculate the power flow through each line

for i in range(branch_data.shape[0]):
    index1 = int(branch_data[i, 0])
    index2 = int(branch_data[i, 1])

    P = 100*branch_data[i, 2] * (angle[index1-1] - angle[index2-1])
    print(f"Power from line {index1} to {index2} is = {P}")




P1 - 300
0.001562*P1**2 + 7.92*P1 + 0.00194*P2**2 + 7.85*P2 + 0.00482*P3**2 + 7.97*P3 + 949
Lagrange_function= 0.001562*P1**2 + 7.92*P1 + 0.00194*P2**2 + 7.85*P2 + 0.00482*P3**2 + 7.97*P3 + θ1*λ4 + λ1*(-P1 + 1800.0*θ1 - 1000.0*θ2 - 800.0*θ3 + 200.0) + λ2*(-P2 - 1000.0*θ1 + 1500.0*θ2 - 500.0*θ3 + 550.0) + λ3*(-P3 - 800.0*θ1 - 500.0*θ2 + 1300.0*θ3 + 100.0) + 949
LP1 = 0.003124*P1 - λ1 + 7.92
LP2 = 0.00388*P2 - λ2 + 7.85
LP3 = 0.00964*P3 - λ3 + 7.97
Lθ1 = 1800.0*λ1 - 1000.0*λ2 - 800.0*λ3 + λ4
Lθ2 = -1000.0*λ1 + 1500.0*λ2 - 500.0*λ3
Lθ3 = -800.0*λ1 - 500.0*λ2 + 1300.0*λ3
Lλ1 = -P1 + 1800.0*θ1 - 1000.0*θ2 - 800.0*θ3 + 200.0
Lλ2 = -P2 - 1000.0*θ1 + 1500.0*θ2 - 500.0*θ3 + 550.0
Lλ3 = -P3 - 800.0*θ1 - 500.0*θ2 + 1300.0*θ3 + 100.0
Lλ4 = θ1
With generator limit implementation {P1: 300.000000000000, P2: 401.035502958580, P3: 148.964497041420, mu1: 0.548817751479290, θ1: 0.0, θ2: -0.0995127044900801, θ3: -0.000609119387399930, λ1: 9.40601775147929, λ2: 9.40601775147929, λ3: 9.40601775147929, λ4: 0